In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Import Library

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

#Evaluation
from sklearn.metrics import auc, roc_curve, roc_auc_score, accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

# Model Tunning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

# Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [48]:
df = pd.read_csv('/content/drive/MyDrive/ISE543/ISE543-FinalProject/Final Project Dataset.csv')
df.head()

,patientID,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD,a1c,income
0,131107,0,49,3.0,1,10.0,0.0,0,0,0,260.0,123.0,80.0,23.10,63.0,65.0,1,3.374776,20195.0
1,318975,1,43,1.0,1,25.0,0.0,0,0,0,201.0,121.0,82.0,23.84,70.0,91.0,0,5.003918,13585.0
2,222774,1,45,1.0,1,1.0,0.0,0,1,0,277.0,140.0,84.0,28.74,69.0,74.0,0,3.654142,27570.0
3,644306,0,63,3.0,1,10.0,0.0,0,1,0,236.0,189.0,103.0,27.91,60.0,74.0,0,4.220056,12217.0
4,101056,1,59,2.0,0,NaN,0.0,0,0,0,237.0,131.5,84.0,24.17,90.0,94.0,1,4.601007,16395.0


# Drop patientID

In [49]:
#Remove id
df.drop(columns='patientID', inplace=True)


# Split

In [50]:
X = df.drop(['TenYearCHD'], axis=1)
y = df['TenYearCHD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 )

# Impute currentSmoker

In [51]:
# fill cigsPerDay with 0
X_train.loc[X_train['currentSmoker'] == 0, 'cigsPerDay'] = X_train.loc[X_train['currentSmoker'] == 0, 'cigsPerDay'].fillna(0)
X_test.loc[X_test['currentSmoker'] == 0, 'cigsPerDay'] = X_test.loc[X_test['currentSmoker'] == 0, 'cigsPerDay'].fillna(0)
# impute median for currentSmoker == 1
median_cigs = X_train[X_train['currentSmoker'] == 1]['cigsPerDay'].median() # parameter to be saved
X_train.loc[X_train['currentSmoker'] == 1,'cigsPerDay'] = X_train.loc[X_train['currentSmoker'] == 1, 'cigsPerDay'].fillna(median_cigs)
X_test.loc[X_test['currentSmoker'] == 1,'cigsPerDay'] = X_test.loc[X_test['currentSmoker'] == 1, 'cigsPerDay'].fillna(median_cigs)

# Impute a1c, glucose, totChol, BMI, heartRate

In [52]:
from sklearn.impute import SimpleImputer

median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')

median_cols = ['a1c', 'glucose', 'totChol', 'BMI', 'heartRate']
mode_cols = ['education', 'BPMeds']

# Impute X_train
X_train[median_cols] = median_imputer.fit_transform(X_train[median_cols])
X_train[mode_cols] = mode_imputer.fit_transform(X_train[mode_cols])

# Impute X_test
X_test[median_cols] = median_imputer.transform(X_test[median_cols])
X_test[mode_cols] = mode_imputer.transform(X_test[mode_cols])


# Convert BPMeds and Education to integer

In [53]:
X_train['BPMeds'] = X_train['BPMeds'].astype(int)
X_test['BPMeds'] = X_test['BPMeds'].astype(int)
X_train['education'] = X_train['education'].astype(int)
X_test['education'] = X_test['education'].astype(int)

# Winsorize Outliers

In [54]:
from scipy.stats.mstats import winsorize

numerical_columns = ['age','cigsPerDay', 'totChol', 'income', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose', 'a1c']

# Dictionary to store the thresholds for each column
winsorize_thresholds = {}

for col in numerical_columns:
    sorted_train_data = np.sort(X_train[col])
    lower_limit_index = int(0.01 * len(sorted_train_data))  # 1% lower limit
    upper_limit_index = int(0.99 * len(sorted_train_data)) - 1  # 99% upper limit
    winsorize_thresholds[col] = (sorted_train_data[lower_limit_index], sorted_train_data[upper_limit_index])

    # Apply to train
    X_train[col] = np.where(X_train[col] < sorted_train_data[lower_limit_index], sorted_train_data[lower_limit_index], X_train[col])
    X_train[col] = np.where(X_train[col] > sorted_train_data[upper_limit_index], sorted_train_data[upper_limit_index], X_train[col])

# Apply to test
for col in numerical_columns:
    lower_limit, upper_limit = winsorize_thresholds[col]
    X_test[col] = np.where(X_test[col] < lower_limit, lower_limit, X_test[col])
    X_test[col] = np.where(X_test[col] > upper_limit, upper_limit, X_test[col])


# Log Transformation

In [55]:
#First log transform
skewed_features = ['age','BMI', 'heartRate', 'totChol', 'glucose']
X_train_transformed = X_train.copy()
for feature in skewed_features:
    X_train_transformed[feature] = np.log1p(X_train_transformed[feature])

In [56]:
#First log transform - test

X_test_transformed = X_test.copy()
for feature in skewed_features:
    X_test_transformed[feature] = np.log1p(X_test_transformed[feature])

# Box-Cox Transformation

In [57]:
# There are still some featrues highly skewed we can consider a more aggresive transformation
# Second box-cox transform
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
skewed_features_second = ['BMI', 'heartRate','glucose']

transformers = {feature: PowerTransformer(method='box-cox', standardize=False) for feature in skewed_features_second}

for feature in skewed_features_second:
    X_train_transformed[feature] = transformers[feature].fit_transform(X_train_transformed[[feature]].values)


In [58]:
# Transform test data using the fitted transformers
for feature in skewed_features_second:
    X_test_transformed[feature] = transformers[feature].transform(X_test_transformed[[feature]].values)


# Drop A1C

In [59]:
#Drop A1C since it has high corr with glucose
X_train_transformed.drop(columns='a1c',inplace=True)
X_test_transformed.drop(columns='a1c',inplace=True)

# Bin Income

Income is extremly imbalanced and has persistence skewness, I decide to bin this feature into categorical data

In [60]:
#Bin train
X_train_bin = X_train_transformed.copy()

# first we obtain bin edges from train
q = X_train_bin['income'].quantile([0.25, 0.5, 0.75]).values

# bin
X_train_bin['income_category'] = pd.qcut(X_train_bin['income'], q=[0, 0.25, 0.5, 0.75, 1], labels=['Low', 'Middle', 'High', 'Very High'])
X_train_bin.drop(columns='income', inplace=True)


In [61]:
#Bin test
X_test_bin = X_test_transformed.copy()

# use cut for fixed bin edges from X_train to ensure consistency
X_test_bin['income_category'] = pd.cut(X_test['income'], bins=[X_test['income'].min()] + q.tolist() + [X_test['income'].max()], labels=['Low', 'Middle', 'High', 'Very High'])
X_test_bin.drop(columns='income', inplace=True)


#Categorize Smoking

In [62]:
X_train_comb = X_train_bin.copy()
X_test_comb = X_test_bin.copy()
def categorize_smoking(row):
    if row['currentSmoker'] == 0:
        return 'non-smoker'
    elif row['cigsPerDay'] <= 5:
        return 'light smoker'
    elif row['cigsPerDay'] <= 20:
        return 'moderate smoker'
    else:
        return 'heavy smoker'
X_train_comb['smoking_status'] = X_train_comb.apply(categorize_smoking, axis=1)
X_train_comb.drop(columns=['currentSmoker', 'cigsPerDay'], inplace=True)

X_test_comb['smoking_status'] = X_test_comb.apply(categorize_smoking, axis=1)
X_test_comb.drop(columns=['currentSmoker', 'cigsPerDay'], inplace=True)

#Categorize BP

In [63]:
def create_bp_category(row):
    if row['prevalentHyp'] == 1:
        return 'Hypertension'
    else:
        if row['sysBP'] < 120 and row['diaBP'] < 80:
            return 'Normal'
        elif (row['sysBP'] >= 120 and row['sysBP'] < 140) or (row['diaBP'] >= 80 and row['diaBP'] < 90):
            return 'Prehypertension'
        elif row['sysBP'] >= 140 or row['diaBP'] >= 90:
            return 'Hypertension'

X_train_comb['bp_category'] = X_train_comb.apply(create_bp_category, axis=1)
X_train_comb = X_train_comb.drop(['sysBP', 'diaBP', 'prevalentHyp'], axis=1)

X_test_comb['bp_category'] = X_test_comb.apply(create_bp_category, axis=1)
X_test_comb = X_test_comb.drop(['sysBP', 'diaBP', 'prevalentHyp'], axis=1)

#Encoding Categorical Data

In [64]:
X_train_encoded = X_train_comb.copy()
X_train_encoded = pd.get_dummies(X_train_encoded, columns=['income_category', 'smoking_status', 'bp_category'])

X_test_encoded = X_test_comb.copy()
X_test_encoded = pd.get_dummies(X_test_encoded, columns=['income_category', 'smoking_status', 'bp_category'])

#Scaling

In [65]:
from sklearn.preprocessing import StandardScaler

X_train_scaled = X_train_encoded.copy()
scaler = StandardScaler()

numerical_features = ['age','totChol', 'BMI', 'heartRate', 'glucose']

X_train_scaled[numerical_features] = scaler.fit_transform(X_train_scaled[numerical_features])


In [66]:
# Scale test
X_test_scaled = X_test_encoded.copy()
X_test_scaled[numerical_features] = scaler.transform(X_test_scaled[numerical_features])


In [67]:
X_test_scaled

,male,age,education,BPMeds,prevalentStroke,diabetes,totChol,BMI,heartRate,glucose,...,income_category_Middle,income_category_High,income_category_Very High,smoking_status_heavy smoker,smoking_status_light smoker,smoking_status_moderate smoker,smoking_status_non-smoker,bp_category_Hypertension,bp_category_Normal,bp_category_Prehypertension
1078,0,-1.150215,1,0,0,0,-0.298757,-0.259511,0.451415,0.000758,...,False,True,False,False,False,True,False,False,False,True
1957,0,-0.731424,1,0,0,0,0.535625,0.152209,1.583443,-0.548769,...,False,False,True,False,True,False,False,False,False,True
2694,1,-0.598151,1,0,0,1,-0.607426,0.641839,1.519882,2.927215,...,False,False,True,False,False,False,True,True,False,False
2002,1,-0.598151,1,0,0,0,-1.948002,-1.308955,-0.703979,-1.743426,...,False,True,False,False,False,True,False,False,True,False
1213,0,1.107921,1,1,0,0,1.353996,0.671140,0.833897,1.781598,...,True,False,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,0,0.259275,2,1,0,0,1.087950,1.401146,-1.227272,-0.449962,...,True,False,False,False,False,False,True,True,False,False
2287,1,1.490660,1,0,0,1,0.621015,1.100776,-0.703979,1.781598,...,False,False,False,False,False,False,True,True,False,False
2567,1,1.107921,1,0,0,0,1.409377,0.840441,-2.321087,1.137919,...,True,False,False,False,False,False,True,True,False,False
700,1,1.673149,1,0,0,0,-1.814769,1.932100,0.037355,0.000758,...,False,False,True,False,False,False,True,True,False,False


#ADASYN

In [68]:
from imblearn.over_sampling import ADASYN

categorical_features_indices = [0, 2, 3, 4, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_scaled, y_train)


# Logistic Regression

In [69]:
def logistic_reg(X_train, y_train, X_test, y_test):
    model = LogisticRegression(class_weight='balanced', max_iter=1000, )
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classif_report)
    print("AUC (Area Under Curve):", auc_score)

logistic_reg(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)

Accuracy: 0.7801047120418848
Confusion Matrix:
 [[567  75]
 [ 93  29]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.88      0.87       642
           1       0.28      0.24      0.26       122

    accuracy                           0.78       764
   macro avg       0.57      0.56      0.56       764
weighted avg       0.77      0.78      0.77       764

AUC (Area Under Curve): 0.6844134620295184


# Decision Trees

In [70]:
def decision_tree(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier(class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classif_report)
    print("AUC (Area Under Curve):", auc_score)

decision_tree(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)

Accuracy: 0.7342931937172775
Confusion Matrix:
 [[526 116]
 [ 87  35]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.82      0.84       642
           1       0.23      0.29      0.26       122

    accuracy                           0.73       764
   macro avg       0.54      0.55      0.55       764
weighted avg       0.76      0.73      0.75       764

AUC (Area Under Curve): 0.5530999438230938


# GaussianNB

doesn't fit for imbalanced data since it assume all features are uniformly distributed

In [71]:
def gaussian_nb(X_train, y_train, X_test, y_test):
  model = GaussianNB()
  model.fit(X_train, y_train)

  #Predict
  y_pred = model.predict(X_test)
  y_prob = model.predict_proba(X_test)[:, 1]

  # Evaluating the model
  accuracy = accuracy_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)
  classif_report = classification_report(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_prob)

  print("Accuracy:", accuracy)
  print("Confusion Matrix:\n", conf_matrix)
  print("Classification Report:\n", classif_report)
  print("AUC (Area Under Curve):", auc_score)

gaussian_nb(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)

Accuracy: 0.6701570680628273
Confusion Matrix:
 [[441 201]
 [ 51  71]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.69      0.78       642
           1       0.26      0.58      0.36       122

    accuracy                           0.67       764
   macro avg       0.58      0.63      0.57       764
weighted avg       0.79      0.67      0.71       764

AUC (Area Under Curve): 0.6915440988713549


# Support Vector Machine

slowest

In [72]:
def support_vector_machine(X_train, y_train, X_test, y_test):
    model = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classif_report)
    print("AUC (Area Under Curve):", auc_score)

support_vector_machine(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)

Accuracy: 0.7958115183246073
Confusion Matrix:
 [[575  67]
 [ 89  33]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.90      0.88       642
           1       0.33      0.27      0.30       122

    accuracy                           0.80       764
   macro avg       0.60      0.58      0.59       764
weighted avg       0.78      0.80      0.79       764

AUC (Area Under Curve): 0.6311858434196415


# Random Forest

In [73]:
def random_forest(X_train, y_train, X_test, y_test):
    # Creating and training the RandomForest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classif_report)
    print("AUC (Area Under Curve):", auc_score)

random_forest(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)


Accuracy: 0.806282722513089
Confusion Matrix:
 [[597  45]
 [103  19]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.93      0.89       642
           1       0.30      0.16      0.20       122

    accuracy                           0.81       764
   macro avg       0.57      0.54      0.55       764
weighted avg       0.76      0.81      0.78       764

AUC (Area Under Curve): 0.6583997242224605


# LightGBM

In [74]:
def lightgbm(X_train, y_train, X_test, y_test):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

  scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

  clf = lgb.LGBMClassifier(
      num_leaves=31,
      max_depth=-1,
      learning_rate=0.05,
      n_estimators=1000,
      scale_pos_weight=scale_pos_weight,
      random_state=42
  )

  # Train the model
  clf.fit(X_train, y_train)

  # Predict on the test set
  y_pred = clf.predict(X_test)
  y_prob = clf.predict_proba(X_test)[:, 1]

  # Evaluating the model
  accuracy = accuracy_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)
  classif_report = classification_report(y_test, y_pred)
  auc_score = roc_auc_score(y_test, y_prob)

  print("Accuracy:", accuracy)
  print("Confusion Matrix:\n", conf_matrix)
  print("Classification Report:\n", classif_report)
  print("AUC (Area Under Curve):", auc_score)

lightgbm(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test)

[LightGBM] [Info] Number of positive: 463, number of negative: 2589
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000489 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1485
[LightGBM] [Info] Number of data points in the train set: 3052, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.151704 -> initscore=-1.721300
[LightGBM] [Info] Start training from score -1.721300
Accuracy: 0.819371727748691
Confusion Matrix:
 [[607  41]
 [ 97  19]]
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.94      0.90       648
           1       0.32      0.16      0.22       116

    accuracy                           0.82       764
   macro avg       0.59      0.55      0.56       764
weighted avg       0.78      0.82      0.79       764

AUC (Area Under Curve): 0

# Voting Ensemble

In [75]:
def voting_ensemble(X_train, y_train, X_test, y_test, estimators):
    model = VotingClassifier(estimators=estimators, voting='soft')
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    classif_report = classification_report(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", classif_report)
    print("AUC (Area Under Curve):", auc_score)

scale_pos_weight = y_train_adasyn.value_counts()[0] / y_train_adasyn.value_counts()[1]
# Create instances of individual models
lr = LogisticRegression(class_weight='balanced', max_iter=1000)
svm = SVC(kernel='sigmoid', class_weight='balanced', probability=True, random_state=42)
rf = RandomForestClassifier(max_depth=10, class_weight='balanced', random_state=42)
lgbm = lgb.LGBMClassifier(
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.05,
    n_estimators=1000,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

# Define the estimators
estimators = [
    ('lr', lr),
    ('rf', rf),
    ('svm', svm),
    ('lgbm', lgbm),
]

#Run
voting_ensemble(X_train_adasyn, y_train_adasyn, X_test_scaled, y_test, estimators)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2679, number of negative: 2595
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1310
[LightGBM] [Info] Number of data points in the train set: 5274, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.507964 -> initscore=0.031857
[LightGBM] [Info] Start training from score 0.031857
Accuracy: 0.7984293193717278
Confusion Matrix:
 [[591  51]
 [103  19]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.92      0.88       642
           1       0.27      0.16      0.20       122

    accuracy                           0.80       764
   macro avg       0.56      0.54      0.54       764
